###  RNN의 개념  
**순환 신경망 = RNN**      
**시계열** -> 과거의 데이터가 현재에 영향을 미친다 -> 현재가 미래에 영향을 미친다     

X를 넣고 -> RNN_cell -> Y가 나옴      
<< 계산 >>     
RNN state라는 개념이 존재 => State를 계산      
H_t = '???'함수(이전 timestep(old step)사용, 현재의 입력값)     
'???' 함수가 모든 RNN과 동일      

In [1]:
# RNN의 기본 신경망은 긴단계의 데이터를 학습할 때 맨 뒤에서는 맨 앞의 정보를 잘 기억하지 못하는 특성이 있음

In [3]:
import tensorflow as tf

In [27]:
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("./mnist/data/", one_hot =True )

In [6]:
# 하이퍼 파라미터
learning_rate = 0.001
total_epoch   = 30
batch_size    = 128

In [8]:
# 변수 
n_input = 28
n_step  = 28
n_hidden = 128
n_class = 10

In [9]:
# 플레이스 홀더
# 가로 픽셀 => n_input //  세로 픽셀  => n_step
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

In [10]:
# 출력 값은 one_ hot 인코딩으로 표현 

In [11]:
# 다중치 & 편향 
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [14]:
#  n_hidden개의 출력 값을 갖는 RNN을 생성 
# 신경망 구성을 위해 BasicRNNCell 사용 
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [28]:
# dynamic_rnn => 두줄로 RNN의 핵심 구조 (셀과 신경망을 만듬) => RNN을 완성 
# 앞에서 생성한 RNN셀 값과 입력값의 자료형을 넣으면 신경망 탄생 !! 
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype = tf.float32)

In [19]:
# 원래는  이렇게 !!
# states = tf.zeros(batch_size)
# for i in range(n_step):
#     outputs,states = cell(X[[:, i]], states)

In [20]:
# 한단계 학습하고 학습상태를 저장하고 그상태를 다음 단계의 입력 상태로 하여 다시 학습한다. => 이렇게 출력 값을 만들어 가는게 RNN 

In [21]:
# RNN_ 
# RNN에서 나온 출력값을 가지고 최종 출력값을 만들자 = > 결과값은  원-핫 인코딩 형태 
# 손실함수 tf.nn.softmax_cross_entropy_with_logits 

# tf.nn.softmax_cross_entropy_with_logits를 사용하려면 
#>> 최종 결괏값이 실측값 Y와 동일한 형태인 [batch_size, n_class]
#>> 같은 형태의 출력 값이 되도록 해야함 
#>> 그러려면 가중치와 편향을 재설정 

In [29]:
# 가중치와 편향 재설정  => 최종 결괏값이 실측값 Y와 동일한 형태여야해서 
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [23]:
# RNN에서 나오는 출력값은 [batch_size, n_step, n_hidden] 형태 
# 은닉층의 출력값을 가중치 W와 같은 형태로 만들어 줘야 
# 행렬곱을 수행하여 원하는 출력값을 얻음 

# **트렌스포스 함수**를 이용해서
# n_step 과 batch_size 차원의 순서를 바꾸고 n_step차원을 제거하여 
# 얻고자하는 최종 출력값의 형태를 만듬
# >>>> ... ? 


In [31]:
# outputs : [ batch_size, n_step, n_hidden ]
# -> [n_step, batch_size, n_hidden]
outputs = tf.transpose(outputs, [1, 0, 2])
# -> [batch_size, n_hidden] 
outputs = outputs[-1]

In [ ]:
# y = X * W + b => 최종 결과 값을 만듬 

In [37]:
model = tf.matmul(outputs, W) + b

In [ ]:
# 모델과 실측 값을 비교하여 손실값을 구하고 
# 신경망을 최적화 하는 함수를 사용하여 신경망 구성을 마무리 

In [34]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [ ]:
# 신경망을 학습 

In [38]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [39]:
total_batch = int(mnist.train.num_examples / batch_size)

In [42]:
for epoch in range(total_batch):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val
        
    print('Epoch:', '%04d' %(epoch + 1), 'Avg.cost=', '{:.3f}'.format(total_cost/ total_batch))
    
print('최적화 완료!')

InvalidArgumentError: logits and labels must be broadcastable: logits_size=[28,10] labels_size=[128,10]
	 [[node softmax_cross_entropy_with_logits_sg_1 (defined at <ipython-input-34-8e53fb506e43>:1) ]]

Original stack trace for 'softmax_cross_entropy_with_logits_sg_1':
  File "C:\Users\JerryKim\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\JerryKim\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\JerryKim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\JerryKim\AppData\Local\Continuum\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\JerryKim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Use

In [36]:

for epoch in range(total_batch):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val
        
    print('Epoch:', '%04d' %(epoch + 1), 'Avg.cost=', '{:3f}'.format(total_cost/ total_batch))
    
print('최적화 완료!')
        
is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
accuracy   = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('정확도:',sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))

InvalidArgumentError: logits and labels must be broadcastable: logits_size=[28,10] labels_size=[128,10]
	 [[node softmax_cross_entropy_with_logits_sg_1 (defined at <ipython-input-34-8e53fb506e43>:1) ]]

Original stack trace for 'softmax_cross_entropy_with_logits_sg_1':
  File "C:\Users\JerryKim\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\JerryKim\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\JerryKim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\JerryKim\AppData\Local\Continuum\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\JerryKim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Use